# Capstone Two: Data Wrangling

## 1. Introduction

The purpose of this data wrangling exercise is to ensur I have sufficient and quality data to use in developing a price prediction model. 

The dataset used in this project was obtained from https://www.kaggle.com/datasets/ahmedshahriarsakib/usa-real-estate-dataset 
The Kaggle command API is !kaggle datasets download -d ahmedshahriarsakib/usa-real-estate-dataset

The definitions of the columns in the dataset are:
status = Housing status - ready for sale or, ready to build
bed  = number of beds
bath = number of bathrooms
acre_lot = Property / Land size in acres
city = name of the city where house is located
state = name of the state
zip_code = postal code of the area
house_size = square footage for the house
prev_sold_date = previous date when the house was last sold
price = Housing price, it is either the current listing price or recently sold price if the house was sold recently

### 1.1 Objectives

Clean the data and ensure it is sufficient to test whether house attributes such as number of bedrooms, bathrooms, as well as location and lot_size correlate with price. Also, if I can predict housing prices based on the features.

## 2. Imported Packages and Libraries

In [ ]:
# Import packages and libraries
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

## 3. Load the Real Estate Data

In [ ]:
# Load the original dataset
df = pd.read_csv('realtor_data.csv')

In [ ]:
# Create a copy of the original dataset
df1 = df.copy()
print(df1.head())

In [ ]:
print(df1.shape)

## 4. Data Definitions: Column Descriptions and Data Types

In [ ]:
df1.info()

In [ ]:
# Counting Unique Values
df1.nunique()

In [ ]:
# Percent of unique values
df1.nunique() / df1.size*100

In [ ]:
# Convert 'zip_code' column to category
df1['zip_code'] = df1['zip_code'].astype('category')

## 5. Data Cleaning

### 5.1 Missing and NaN values

In [ ]:
missing = pd.concat([df1.isna().sum(), 100 * df1.isna().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=False)

The prev_sold_date feature has the highest numbe rof missing values. This feature does not play an important role in meeting this project's objective but I would not drop it for now. Acre_lot and house_size are important to meeting my objective; these fatures will be dropped because I believe I have sufficient data.

In [ ]:
# Drop null values from 'acre_lot' and 'house_size' columns
df1.dropna(subset=['acre_lot', 'house_size'], inplace=True)

print(df1.shape)

In [ ]:
# Find missing values in the DataFrame
missing_values = df1.isna()

#Display the count of missing values
print(missing_values.sum())

Earlier, there were zero entries in the in the 'price' and 'acre_lot' columns. Drop those values from the price column.

In [ ]:
# Count the number of values in the 'price' column that are either 0.0 or NaN
price_zero_or_nan = df1['price'].isin([0, 0.0, float('nan')]).sum()

print(f"Number of values in the 'price' column that are either 0.0 or NaN:", price_zero_or_nan)

In [ ]:
missing_bed_bath = df1[['bed', 'bath']].isna().sum(axis=1)
missing_bed_bath.value_counts()/len(missing_bed_bath) * 100

Over 96% of 'bed' and 'bath' data have no missing values, 2% are missing both values, and about 1.5% have a value missing either in the 'bed' or 'bath' column.

In [ ]:
# Iterate through the 'price' column and drop values equal to 0 or 0.0
drop_indices = []
for index, value in df1['price'].items():
    if value == 0 or value == 0.0:
        drop_indices.append(index)

# Drop rows in the drop_indices list
df1.drop(drop_indices, inplace=True, errors='ignore')

# Drop rows where both 'bed' and 'bath' columns have missing values
df1.dropna(subset=['bed', 'bath'], inplace=True)

# Reset index after dropping rows
df1.reset_index(drop=True, inplace=True)

#Display results
print(df1.shape)

The column with the date when a house was previously sold has a lot missing data. I would not want to drop theses values because it may contain other useful data. Therefore I would impute with the current date.

In [ ]:
# Convert the 'prev_sold_date' to datetime
df1['prev_sold_date'] = pd.to_datetime(df1['prev_sold_date'], errors='coerce')

df1['prev_sold_date'] = df1['prev_sold_date'].dt.strftime("%y - %m - %d")

# Fill missing values in 'prev_sold_date' column with today's date
df1['prev_sold_date'].fillna(pd.Timestamp.today().date(), inplace=True)

In [ ]:
# Determine missingness
sorted_df1 = df1.sort_values(by='price')
msno.matrix(sorted_df1)
plt.show()

In [ ]:
df1.describe().T

### 5.2 Handling Duplicate Rows

In [ ]:
duplicate_df1 = df1[df1.duplicated()]
duplicate_df1

In [ ]:
df1.drop_duplicates(subset=["bed", "bath", "acre_lot", "city", "state", "zip_code", "house_size", "price"], inplace=True)

print(df1.shape)

In [ ]:
df1.describe().T

### 5.3 Handling Outliers

In [ ]:
# Handling Outliers
numeric_cols = ['bed', 'bath', 'acre_lot', 'house_size', 'price']
seventy_fifth = df1[numeric_cols].quantile(0.75)
twenty_fifth = df[numeric_cols].quantile(0.25)
df1_iqr = seventy_fifth - twenty_fifth

upper_threshold = seventy_fifth + (1.5 * df1_iqr)

lower_threshold = twenty_fifth - (1.5 * df1_iqr)

print('Upper_threshold: ')
print(upper_threshold)

print('\nLower_threshold: ')
print(lower_threshold)

In [ ]:
count_less_than_536000 = (df1['price'] <= 536000).sum()
print(count_less_than_536000)

In [ ]:
count_more_than_1520000 = (df1['price'] > 1520000).sum()
print(count_more_than_1520000)

In [ ]:
null_acre_lot = (df1['acre_lot'] == 0.0).sum()
print(null_acre_lot)

In [ ]:
price_less_10000 = (df1['price'] <= 20000).sum()
print(price_less_10000)

## 5. Conclusion